* https://github.com/mahyargm/Auto-Tune-Detection

# Get *.wav Vocal Songs

* https://pixabay.com/music/search/genre/vocal/

* https://slooply.com/samples/category/hip-hop

# METHOD #1: Librosa

Code Extrapolated/Derived from Tutorial:
* https://thewolfsound.com/how-to-auto-tune-your-voice-with-python/

With Help of ChatGPT 4o
* https://chatgpt.com/share/676178b2-f948-800d-98c5-b14a34374e88



In [1]:
# !pip install librosa soundfile numpy scipy

!pip install librosa==0.9.2 soundfile==0.12.1 scipy==1.8.1 numpy==1.22.2 matplotlib==3.5.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.3/214.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 26.4 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
  Attempting uninstall: scipy
    Found existing installation: scipy 1.13.1
    Uninstalling scipy-1.13.1:
      Successfully uninstalled scipy-1.13.1
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.8.0
    Uninstalling matplotlib-3.8.0:
      Successfully uninstalled matplotlib-3.8.0
  Attempting uninstall: librosa
    Found existing installation: librosa 0.10.2.post1
    Uninstalling librosa-0.10.2.post1:
      Succe

**[RESTART]** Required

In [1]:
import librosa
import librosa.display
import numpy as np
import soundfile as sf
import scipy.signal as sig
import matplotlib.pyplot as plt
import json

In [2]:
SEMITONES_IN_OCTAVE = 12

def degrees_from(scale: str):
    """Return pitch classes (degrees) corresponding to a given scale."""
    degrees = librosa.key_to_degrees(scale)
    degrees = np.concatenate((degrees, [degrees[0] + SEMITONES_IN_OCTAVE]))
    return degrees

def closest_pitch_from_scale(f0, scale):
    """Round pitch to the closest note in a given scale."""
    if np.isnan(f0):
        return np.nan
    degrees = degrees_from(scale)
    midi_note = librosa.hz_to_midi(f0)
    degree = midi_note % SEMITONES_IN_OCTAVE
    degree_id = np.argmin(np.abs(degrees - degree))
    degree_diff = degree - degrees[degree_id]
    midi_note -= degree_diff
    return librosa.midi_to_hz(midi_note)

def map_pitch_to_scale(f0, scale):
    """Map pitch array to closest notes in the scale with smoothing."""
    corrected_pitch = np.zeros_like(f0)
    for i in range(len(f0)):
        corrected_pitch[i] = closest_pitch_from_scale(f0[i], scale)
    smoothed = sig.medfilt(corrected_pitch, kernel_size=11)
    nan_mask = np.isnan(f0)
    smoothed[nan_mask] = np.nan
    return smoothed

def cents_difference(freq1, freq2):
    """Calculate the pitch difference in cents."""
    if np.isnan(freq1) or np.isnan(freq2) or freq1 <= 0 or freq2 <= 0:
        return np.nan
    return 1200 * np.log2(freq2 / freq1)

def analyze_autotune(input_file, scale="C:min", threshold_cents=50.0, output_json=None):
    print(f"Loading file: {input_file}")
    y, sr = librosa.load(input_file, sr=None, mono=True)

    frame_length = 2048
    hop_length = frame_length // 4
    fmin = librosa.note_to_hz('C2')
    fmax = librosa.note_to_hz('C7')

    f0, _, _ = librosa.pyin(y, sr=sr, fmin=fmin, fmax=fmax, frame_length=frame_length, hop_length=hop_length)
    corrected_f0 = map_pitch_to_scale(f0, scale)
    pitch_differences = np.array([cents_difference(orig, corr) for orig, corr in zip(f0, corrected_f0)])

    significant_correction_mask = np.abs(pitch_differences) > threshold_cents
    times = librosa.frames_to_time(np.arange(len(f0)), sr=sr, hop_length=hop_length)

    segments = []
    currently_in_segment = False
    segment_start = None

    for i, val in enumerate(significant_correction_mask):
        if val and not currently_in_segment:
            currently_in_segment = True
            segment_start = times[i]
        elif not val and currently_in_segment:
            currently_in_segment = False
            segments.append((segment_start, times[i]))
    if currently_in_segment:
        segments.append((segment_start, times[-1]))

    total_duration = len(y) / sr
    corrected_frames_count = np.sum(significant_correction_mask)
    corrected_percentage = (corrected_frames_count / len(f0)) * 100.0 if len(f0) > 0 else 0.0

    report = {
        "file": input_file,
        "scale": scale,
        "threshold_cents": threshold_cents,
        "total_duration_s": total_duration,
        "corrected_percentage": corrected_percentage,
        "corrected_segments": [{"start_s": float(s), "end_s": float(e)} for s, e in segments]
    }

    print("\nAutotune Analysis Report")
    print("========================")
    print(f"Total Duration: {total_duration:.2f} seconds")
    print(f"Corrected Pitch Frames: {corrected_percentage:.2f}% of track")
    if segments:
        print("\nSignificant Pitch Corrections Detected:")
        for seg in report['corrected_segments']:
            print(f" - {seg['start_s']:.2f}s to {seg['end_s']:.2f}s")
    else:
        print("No significant pitch corrections detected.")

    if output_json:
        with open(output_json, "w") as f:
            json.dump(report, f, indent=2)
        print(f"\nJSON Report saved to {output_json}")

    return report




Saving 20_SS_ALV_Haces_Libs_Vocals_92_Amin_Wet_01.wav to 20_SS_ALV_Haces_Libs_Vocals_92_Amin_Wet_01.wav
Loading file: 20_SS_ALV_Haces_Libs_Vocals_92_Amin_Wet_01.wav

Autotune Analysis Report
Total Duration: 31.30 seconds
Corrected Pitch Frames: 58.62% of track

Significant Pitch Corrections Detected:
 - 0.34s to 1.67s
 - 1.68s to 1.75s
 - 1.76s to 1.78s
 - 1.85s to 1.87s
 - 2.07s to 2.08s
 - 2.14s to 2.15s
 - 2.16s to 2.19s
 - 2.21s to 2.23s
 - 2.31s to 2.51s
 - 2.53s to 2.57s
 - 2.58s to 2.59s
 - 2.76s to 5.14s
 - 5.25s to 5.41s
 - 5.43s to 5.46s
 - 5.50s to 5.53s
 - 5.54s to 5.72s
 - 5.76s to 5.90s
 - 5.96s to 6.07s
 - 6.08s to 6.10s
 - 6.18s to 6.25s
 - 6.26s to 6.27s
 - 6.32s to 6.33s
 - 6.34s to 6.42s
 - 6.44s to 6.47s
 - 6.68s to 6.69s
 - 6.71s to 6.72s
 - 6.76s to 6.78s
 - 6.80s to 6.83s
 - 6.86s to 6.88s
 - 6.91s to 6.92s
 - 7.08s to 7.12s
 - 7.15s to 7.51s
 - 7.55s to 8.13s
 - 8.20s to 8.51s
 - 8.57s to 8.90s
 - 8.92s to 8.99s
 - 9.01s to 9.67s
 - 9.69s to 9.90s
 - 9.93s to 10

## Upload Song #1

In [3]:
# Colab-Friendly Execution
input_audio = "input_audio.wav"  # Replace with your file name
uploaded = False

try:
    from google.colab import files
    uploaded_file = files.upload()
    input_audio = list(uploaded_file.keys())[0]
    uploaded = True
except ImportError:
    print("Google Colab not detected. Ensure the input file is present in the working directory.")

if uploaded:
    output_report = "autotune_report.json"
    report = analyze_autotune(input_audio, scale="C:min", threshold_cents=50.0, output_json=output_report)

Saving ho-ho-ho-and-away-we-go-274863.mp3 to ho-ho-ho-and-away-we-go-274863.mp3
Loading file: ho-ho-ho-and-away-we-go-274863.mp3

Autotune Analysis Report
Total Duration: 77.23 seconds
Corrected Pitch Frames: 13.96% of track

Significant Pitch Corrections Detected:
 - 1.13s to 1.15s
 - 1.64s to 1.67s
 - 1.72s to 1.73s
 - 1.75s to 1.76s
 - 1.90s to 1.93s
 - 2.09s to 2.10s
 - 2.18s to 2.22s
 - 2.24s to 2.25s
 - 2.26s to 2.27s
 - 2.88s to 2.90s
 - 2.93s to 2.95s
 - 2.97s to 2.98s
 - 3.10s to 3.16s
 - 3.17s to 3.18s
 - 3.23s to 3.24s
 - 3.33s to 3.34s
 - 3.48s to 3.49s
 - 3.50s to 3.56s
 - 3.71s to 3.77s
 - 3.89s to 3.90s
 - 3.94s to 4.21s
 - 4.27s to 4.44s
 - 4.78s to 4.79s
 - 4.81s to 4.82s
 - 4.85s to 4.93s
 - 5.08s to 5.10s
 - 5.15s to 5.16s
 - 5.28s to 5.32s
 - 5.65s to 5.67s
 - 6.01s to 6.02s
 - 6.03s to 6.04s
 - 6.07s to 6.08s
 - 6.18s to 6.25s
 - 6.41s to 6.42s
 - 6.44s to 6.45s
 - 6.58s to 6.59s
 - 6.60s to 6.66s
 - 6.67s to 6.74s
 - 6.89s to 6.90s
 - 6.91s to 6.92s
 - 6.97s to 6.

## Upload Song #2

In [4]:
# Colab-Friendly Execution
input_audio = "input_audio.wav"  # Replace with your file name
uploaded = False

try:
    from google.colab import files
    uploaded_file = files.upload()
    input_audio = list(uploaded_file.keys())[0]
    uploaded = True
except ImportError:
    print("Google Colab not detected. Ensure the input file is present in the working directory.")

if uploaded:
    output_report = "autotune_report.json"
    report = analyze_autotune(input_audio, scale="C:min", threshold_cents=50.0, output_json=output_report)


Saving happy-birthday-258264.mp3 to happy-birthday-258264.mp3
Loading file: happy-birthday-258264.mp3

Autotune Analysis Report
Total Duration: 65.91 seconds
Corrected Pitch Frames: 12.05% of track

Significant Pitch Corrections Detected:
 - 1.64s to 1.66s
 - 2.19s to 2.22s
 - 3.25s to 3.31s
 - 4.08s to 4.10s
 - 4.99s to 5.25s
 - 5.27s to 5.75s
 - 5.97s to 5.98s
 - 6.19s to 6.30s
 - 6.91s to 6.93s
 - 7.30s to 7.31s
 - 9.71s to 9.85s
 - 9.86s to 9.87s
 - 11.19s to 11.20s
 - 11.23s to 11.24s
 - 11.31s to 11.33s
 - 12.26s to 12.29s
 - 12.31s to 12.33s
 - 12.53s to 12.56s
 - 12.90s to 12.91s
 - 13.33s to 13.35s
 - 14.79s to 14.80s
 - 19.82s to 19.97s
 - 20.12s to 20.32s
 - 20.61s to 21.12s
 - 21.26s to 21.40s
 - 21.43s to 22.21s
 - 22.34s to 22.41s
 - 23.56s to 23.57s
 - 23.73s to 23.75s
 - 24.04s to 24.07s
 - 24.22s to 24.24s
 - 24.66s to 24.68s
 - 25.09s to 25.10s
 - 25.41s to 25.43s
 - 25.47s to 25.50s
 - 26.35s to 26.39s
 - 26.46s to 26.49s
 - 27.23s to 27.32s
 - 28.93s to 28.94s
 - 29

## Upload Song #3

In [ ]:
# Colab-Friendly Execution
input_audio = "input_audio.wav"  # Replace with your file name
uploaded = False

try:
    from google.colab import files
    uploaded_file = files.upload()
    input_audio = list(uploaded_file.keys())[0]
    uploaded = True
except ImportError:
    print("Google Colab not detected. Ensure the input file is present in the working directory.")

if uploaded:
    output_report = "autotune_report.json"
    report = analyze_autotune(input_audio, scale="C:min", threshold_cents=50.0, output_json=output_report)

## Upload Song #4

In [ ]:
# Colab-Friendly Execution
input_audio = "input_audio.wav"  # Replace with your file name
uploaded = False

try:
    from google.colab import files
    uploaded_file = files.upload()
    input_audio = list(uploaded_file.keys())[0]
    uploaded = True
except ImportError:
    print("Google Colab not detected. Ensure the input file is present in the working directory.")

if uploaded:
    output_report = "autotune_report.json"
    report = analyze_autotune(input_audio, scale="C:min", threshold_cents=50.0, output_json=output_report)

## Upload Song #5

In [ ]:
# Colab-Friendly Execution
input_audio = "input_audio.wav"  # Replace with your file name
uploaded = False

try:
    from google.colab import files
    uploaded_file = files.upload()
    input_audio = list(uploaded_file.keys())[0]
    uploaded = True
except ImportError:
    print("Google Colab not detected. Ensure the input file is present in the working directory.")

if uploaded:
    output_report = "autotune_report.json"
    report = analyze_autotune(input_audio, scale="C:min", threshold_cents=50.0, output_json=output_report)

# METHOD #2: Auto-Tune-Detection (Not Working Yet)

Needs Copyright Protected ATD-Dataset (Training Dataset) at:
* https://zenodo.org/records/10695885

Website
* https://mahyargm.github.io/Auto-Tune-Detection/

Paper
* https://arxiv.org/html/2403.05380

Related Papers (Scroll Down)
* https://www.semanticscholar.org/paper/Spectrogram-Based-Detection-of-Auto-Tuned-Vocals-in-Gohari-Bestagini/1aabf7b71f849969bccdcec3b08659cf39ade52d

In [1]:
!git clone https://github.com/mahyargm/Auto-Tune-Detection.git

Cloning into 'Auto-Tune-Detection'...
remote: Enumerating objects: 95, done.
remote: Counting objects: 100% (41/41), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 95 (delta 17), reused 20 (delta 7), pack-reused 54 (from 1)
Receiving objects: 100% (95/95), 104.93 MiB | 23.73 MiB/s, done.
Resolving deltas: 100% (20/20), done.


In [2]:
%cd Auto-Tune-Detection
!ls -altr

/content/Auto-Tune-Detection
total 53836
drwxr-xr-x 1 root root     4096 Dec 17 12:17 ..
-rw-r--r-- 1 root root        8 Dec 17 12:17 .gitignore
-rw-r--r-- 1 root root     5738 Dec 17 12:17 ATD_Validation_SM.py
-rw-r--r-- 1 root root     7275 Dec 17 12:17 ATD_Validation.py
-rw-r--r-- 1 root root     3520 Dec 17 12:17 ATD_Training.py
-rw-r--r-- 1 root root     8494 Dec 17 12:17 ATD_TrainingCL.py
-rw-r--r-- 1 root root     6975 Dec 17 12:17 ATD_Data_Preparaton.py
-rw-r--r-- 1 root root 54990384 Dec 17 12:17 Auto_Tune_Detection-master.zip
drwxr-xr-x 3 root root     4096 Dec 17 12:17 Dataset_Creation_Tools
drwxr-xr-x 3 root root     4096 Dec 17 12:17 B02_models
-rw-r--r-- 1 root root     8006 Dec 17 12:17 Runtime_Evaluation.py
-rw-r--r-- 1 root root     3236 Dec 17 12:17 README.md
-rw-r--r-- 1 root root    13372 Dec 17 12:17 RawNet2.py
-rw-r--r-- 1 root root     5594 Dec 17 12:17 baseline_training.py
-rw-r--r-- 1 root root     6228 Dec 17 12:17 baseline_data_preparaton.py
-rw-r--r-- 1 root

In [3]:
# !cat requirements.txt

In [4]:
!sed -i 's/torch==2.2.0/torch==2.0.1/' requirements.txt
!sed -i 's/torchaudio==2.2.0/torchaudio==2.0.1/' requirements.txt
!sed -i 's/torchvision==0.15.1/torchvision==0.15.1/' requirements.txt
!cat requirements.txt

fastai==2.7.12
librosa==0.9.2
matplotlib==3.7.1
numpy==1.22.2
opencv_python==4.7.0.72
Pillow==10.2.0
psola==0.0.1
PyYAML==6.0.1
soundfile==0.12.1
torch==2.0.1
torchaudio==2.0.1
torchvision==0.15.1
tqdm==4.65.0

In [5]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 3.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchaudio to determine which version is compatible with other requirements. This could take a while.
ERROR: Cannot install -r requirements.txt (line 1), -r requirements.txt (line 11) and torch==2.0.1 because these package versions have conflicting dependencies.

The conflict is caused by:
    The user requested torch==2.0.1
    fastai 2.7.12 depends on torch<2.1 and >=1.7
    torchaudio 2.0.1 depends on torch==2.0.0

To fix this you could try to:
1. loosen the range of package versions you've specified
2. remove package versions to allow pip to attempt to solve the dependency conflict

ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/topics/dependency-resolution/#dealing-with-dependency-conflicts


# NOTES

Perplexity.ai Research Chat:

* https://www.perplexity.ai/search/any-free-or-open-source-framew-_h76KvZKR9CcnK4hrlVImA

# PROMPT: any free or open source frameworks/website, python libraries or AI models to detect autotune in music audio files

Several open-source frameworks and Python libraries are available for detecting autotune in music audio files:

## Auto-Tune Detector (ATD)

This is a deep learning-based tool that utilizes spectrogram techniques to identify Auto-Tuned vocals in music recordings[1]. Key features include:

- Python-based implementation
- Uses deep learning and spectrogram analysis
- Available on GitHub (https://github.com/mahyargm/Auto-Tune-Detection)
- Requires Python 3.8 or higher

To use ATD:

1. Install dependencies:
```bash
pip install -r requirements.txt
```

2. Prepare the dataset:
```python
python ATD_Data_Preparaton.py <dataset_directory> <Processed_dataset_dir>
```

## Spectrogram-Based Auto-Tune Detection

This approach leverages triplet networks for detecting Auto-Tuned songs[2][7]. Key aspects include:

- Uses mel-spectrograms as input
- Employs triplet networks to distinguish between Auto-Tuned and genuine vocals
- Outperforms Rawnet2, an end-to-end model used for audio forensics

## Python Auto-Tune Implementation

While not specifically for detection, this Python-based auto-tune implementation can be adapted for analysis[3]:

- Uses PYIN algorithm for pitch tracking
- Employs PSOLA algorithm for pitch shifting
- Available on GitHub (https://github.com/JanWilczek/python-auto-tune)

To use:
```bash
python auto_tune.py "audio_file.wav" -c scale --scale C:min --plot
```

## Musiio AI Autotune Classifier

Musiio has developed an AI-based autotune classifier[4] that:

- Detects the presence and extent of autotune in tracks
- Classifies autotune into four categories: None, Low, Medium, High
- Achieves over 90% accuracy

While not open-source, it demonstrates the capabilities of AI in autotune detection.

## Librosa

Although not specifically designed for autotune detection, Librosa is a powerful Python library for music and audio analysis that can be used as a foundation for building an autotune detector[3].

By combining these tools and approaches, you can develop a robust system for detecting autotune in music audio files. The choice between deep learning models like ATD and traditional signal processing techniques will depend on your specific requirements and the characteristics of your audio dataset.

Citations:
[1] https://github.com/mahyargm/Auto-Tune-Detection
[2] https://arxiv.org/html/2403.05380v1
[3] https://thewolfsound.com/how-to-auto-tune-your-voice-with-python/
[4] https://blog.musiio.com/posts/explaining-autotune-classifier
[5] https://github.com/alexcrist/autotone
[6] https://www.youtube.com/watch?v=JLQF5fDkgYo
[7] http://arxiv.org/abs/2403.05380
[8] https://forum.bela.io/d/1380-pitch-correction-harmonization-and-autotune-effects
[9] https://www.youtube.com/watch?v=HvKpBbHj5cY
[10] http://tombaran.info/autotalent.html

# PROMPT: What are the best Python libraries for detecting Auto-Tune in music

PROMPT:```
What are the best Python libraries for detecting Auto-Tune in music
```

Several Python libraries and tools can be used to detect Auto-Tune in music:

## Auto-Tune Detector (ATD)

This is a deep learning-based tool specifically designed for identifying Auto-Tuned vocals in music recordings[1]. Key features include:

- Utilizes spectrogram-based techniques
- Implemented in Python
- Requires Python 3.8 or higher
- Available on GitHub

To use ATD:

1. Install dependencies:
```bash
pip install -r requirements.txt
```

2. Prepare the dataset:
```python
python ATD_Data_Preparaton.py <dataset_directory> <Processed_dataset_dir>
```

## Librosa

While not specifically designed for Auto-Tune detection, Librosa is a powerful Python library for music and audio analysis that can be adapted for this purpose[2][5]. It provides functions for:

- Onset detection
- Pitch tracking
- Spectrogram analysis

These features can be combined to create an Auto-Tune detection algorithm.

## Aubio

Aubio is another Python library for audio signal processing that can be useful in detecting Auto-Tune[2][5]. It offers:

- Pitch detection algorithms
- Onset detection
- Beat tracking

These functionalities can be leveraged to analyze pitch changes characteristic of Auto-Tune.

## PYIN and PSOLA

These algorithms, while not libraries themselves, are often implemented in Python packages and can be used for Auto-Tune detection[4]:

- PYIN (Probabilistic YIN) for pitch tracking
- PSOLA (Pitch Synchronous Overlap and Add) for pitch shifting analysis

## Custom Implementation

For those interested in building their own Auto-Tune detector, Eric-D-Stevens has created a basic Python implementation of the Auto-Tune patent[9]. This project uses:

- Auto-correlation for pitch detection
- Resampling for pitch correction

While these libraries and tools provide a strong foundation for detecting Auto-Tune, it's important to note that creating a robust Auto-Tune detector is a complex task. It may require combining multiple techniques and potentially implementing custom algorithms to achieve high accuracy across various music styles and Auto-Tune intensities.

Citations:
[1] https://github.com/mahyargm/Auto-Tune-Detection
[2] https://www.youtube.com/watch?v=aMMI0nAKgI0
[3] https://neptune.ai/blog/best-tools-for-model-tuning-and-hyperparameter-optimization
[4] https://thewolfsound.com/how-to-auto-tune-your-voice-with-python/
[5] https://www.freecodecamp.org/news/use-python-to-detect-music-onsets/
[6] https://www.run.ai/guides/automl/automl-python
[7] https://www.youtube.com/watch?v=JLQF5fDkgYo
[8] https://gist.github.com/0xdevalias/7f4a5c31758e04aea5c2f5520e53accb
[9] https://github.com/Eric-D-Stevens/Auto_Tune
[10] https://www.reddit.com/r/learnpython/comments/n9f3bf/making_autotune_software_in_python/